## Introduction

In this part, we will use Colab's free GPU for computation, and we will explore whether a more complex neural network framework might be able to learn the essential features of EEG signals to complete our classification task. (Because of time, we mainly test the network structure instead of innovating the network structure; at the same time, if the accuracy of a network with default parameters exceeds half, we will consider adjusting the parameters to obtain better performance)

More specifically, we will perform the following tasks:

1. Binary classification task using the original simple network.
2. Binary classification task using complex neural network.
3. Multi-classification task using the original simple network.
4. Multi-classification tasks using complex neural networks.
5. Test the neural network model of the BP+CNN.

### Import packages


In [1]:
import sys
from google.colab import drive

# change to your address
drive.mount('/content/gdrive') 
sys.path.append('/content/gdrive/MyDrive/UMONS/Notebooks_DL/dl_models/arl-eegmodels/')
sys.path.append('/content/gdrive/MyDrive/UMONS/Notebooks_DL/pipelines/')

Mounted at /content/gdrive


In [25]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, Dense, Flatten, GRU, Dense, LSTM, RNN, RepeatVector, TimeDistributed, SimpleRNN, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Activation, Dropout, SpatialDropout1D, SpatialDropout2D, BatchNormalization
from tensorflow.keras.layers import Flatten, InputSpec, Layer, Concatenate, AveragePooling2D, MaxPooling2D, Reshape, Permute
from tensorflow.keras.layers import Conv2D, SeparableConv2D, DepthwiseConv2D, ConvLSTM2D, LayerNormalization
from tensorflow.keras.layers import Lambda, AveragePooling1D, Attention, Dot, Add, Multiply
from tensorflow.keras.constraints import max_norm, unit_norm 
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, recall_score, precision_score

### Import NN models

Complex models 1(in folder dl_models/arl-eegmodels/):

In [ ]:
# in folder arl-eegmodels/
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet

Complex models 2(in folder dl_models/EEG-ITNet/):

In [ ]:
Fs = 125          # Sampling frequency
n_ff = [2,4,8]    # Number of frequency filters for each inception module of EEG-ITNet
n_sf = [1,1,1,1]    # Number of spatial filters in each frequency sub-band of EEG-ITNet


def SQ (in_tensor):
    return tf.squeeze(in_tensor, axis=1)

def Select(in_tensor):
    return in_tensor[:,-1,:]

def Network(Chans, Samples, Net_name, drop_rate, out_type = 'single',out_class = 27):
  
    Input_block = Input(shape = (Chans, Samples, 1))
    
    #======================================================================================== 
    # EEG-Inception
    #========================================================================================
    
    if Net_name == 1:
        
        block1 = Conv2D(8, (1, 64), padding='same')(Input_block)
        block1 = BatchNormalization()(block1)
        block1 = Activation('elu')(block1)
        block1 = Dropout(drop_rate)(block1)

        block1 = DepthwiseConv2D((Chans, 1), padding='valid', depth_multiplier = 2)(block1)
        block1 = BatchNormalization()(block1)
        block1 = Activation('elu')(block1)
        block1 = Dropout(drop_rate)(block1)

        #================================

        block2 = Conv2D(8, (1, 32), padding='same')(Input_block)
        block2 = BatchNormalization()(block2)
        block2 = Activation('elu')(block2)
        block2 = Dropout(drop_rate)(block2)

        block2 = DepthwiseConv2D((Chans, 1), padding='valid', depth_multiplier = 2)(block2)
        block2 = BatchNormalization()(block2)
        block2 = Activation('elu')(block2)
        block2 = Dropout(drop_rate)(block2)

        #================================

        block3 = Conv2D(8, (1, 16), padding='same')(Input_block)
        block3 = BatchNormalization()(block3)
        block3 = Activation('elu')(block3)
        block3 = Dropout(drop_rate)(block3)

        block3 = DepthwiseConv2D((Chans, 1), padding='valid', depth_multiplier = 2)(block3)
        block3 = BatchNormalization()(block3)
        block3 = Activation('elu')(block3)
        block3 = Dropout(drop_rate)(block3)

        #================================

        block = Concatenate(axis = -1)([block1, block2, block3])
        block = AveragePooling2D((1, 4))(block)

        #================================

        block1_1 = Conv2D(8, (1, 16), padding='same')(block)
        block1_1 = BatchNormalization()(block1_1)
        block1_1 = Activation('elu')(block1_1)
        block1_1 = Dropout(drop_rate)(block1_1)

        #================================

        block2_1 = Conv2D(8, (1, 8), padding='same')(block)
        block2_1 = BatchNormalization()(block2_1)
        block2_1 = Activation('elu')(block2_1)
        block2_1 = Dropout(drop_rate)(block2_1)

        #================================

        block3_1 = Conv2D(8, (1, 4), padding='same')(block)
        block3_1 = BatchNormalization()(block3_1)
        block3_1 = Activation('elu')(block3_1)
        block3_1 = Dropout(drop_rate)(block3_1)

        #================================

        block_new = Concatenate(axis = -1)([block1_1, block2_1, block3_1])
        block_new = AveragePooling2D((1, 2))(block_new)

        block_new = Conv2D(12, (1, 8), padding='same')(block_new)
        block_new = BatchNormalization()(block_new)
        block_new = Activation('elu')(block_new)
        block_new = Dropout(drop_rate)(block_new)

        block_new = AveragePooling2D((1, 2))(block_new)

        block_new = Conv2D(6, (1, 4), padding='same')(block_new)
        block_new = BatchNormalization()(block_new)
        block_new = Activation('elu')(block_new)
        block_new = Dropout(drop_rate)(block_new)

        block_new = AveragePooling2D((1, 2))(block_new)

        embedded = Flatten()(block_new)
        out = Dense(out_class, activation = 'softmax')(embedded)
    
    #========================================================================================
    # EEGNet
    #========================================================================================
    
    elif Net_name == 2:

        block = Conv2D(8, (1, 64), use_bias = False, activation = 'linear', padding='same',
                       name = 'Spectral_filter')(Input_block)
        block = BatchNormalization()(block)
        block = DepthwiseConv2D((Chans, 1), use_bias = False, padding='valid', depth_multiplier = 2, activation = 'linear',
                                 depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1),
                                name = 'Spatial_filter')(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)

        #================================

        block = AveragePooling2D((1, 4))(block)
        block = Dropout(drop_rate)(block)

        block = SeparableConv2D(16, (1, 16), use_bias = False, activation = 'linear', padding = 'same')(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = AveragePooling2D((1, 8))(block)
        block = Dropout(drop_rate)(block)
        embedded = Flatten()(block)

        out = Dense(out_class, activation = 'softmax', kernel_constraint = max_norm(0.25))(embedded)
    
    #========================================================================================   
    # EEG-TCNet
    #========================================================================================
    
    elif Net_name == 3:
        
        drop_rate = [drop_rate, drop_rate-0.1]
        
        block = Conv2D(8, (1, 64), use_bias = False, activation = 'linear', padding='same',
                       name = 'Spectral_filter')(Input_block)
        block = BatchNormalization()(block)
        block = DepthwiseConv2D((Chans, 1), use_bias = False, padding='valid', depth_multiplier = 2, activation = 'linear',
                                 depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1),
                                name = 'Spatial_filter')(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)

        #================================

        block = AveragePooling2D((1, 4))(block)
        block = Dropout(drop_rate[0])(block)

        block = SeparableConv2D(16, (1, 16), use_bias = False, activation = 'linear', padding = 'same')(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = AveragePooling2D((1, 8))(block)
        block = Dropout(drop_rate[0])(block)

        #================================

        block_in = Lambda(SQ)(block)

        paddings = tf.constant([[0,0], [2,0], [0,0]])
        block = tf.pad(block_in, paddings, "CONSTANT")
        block = Conv1D(16, 3, dilation_rate=1)(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate[1])(block) 
        block = tf.pad(block, paddings, "CONSTANT")
        block = Conv1D(16, 3, dilation_rate=1)(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate[1])(block) 
        block_in = Conv1D(16, 1)(block_in)
        block_out = Add()([block_in, block])


        paddings = tf.constant([[0,0], [4,0], [0,0]])
        block = tf.pad(block_out, paddings, "CONSTANT")
        block = Conv1D(16, 3, dilation_rate=2)(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate[1])(block) 
        block = tf.pad(block, paddings, "CONSTANT")
        block = Conv1D(16, 3, dilation_rate=2)(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate[1])(block) 
        block_out = Conv1D(16, 1)(block_out)
        block = Add()([block_out, block])

        embedded = Lambda(Select)(block)

        out = Dense(out_class, activation = 'softmax', kernel_constraint = max_norm(0.25))(embedded)

    #========================================================================================   
    # EEG-ITNet
    #========================================================================================  
    
    else:    
      
        block1 = Conv2D(n_ff[0], (1, 16), use_bias = False, activation = 'linear', padding='same',
                       name = 'Spectral_filter_1')(Input_block)
        block1 = BatchNormalization()(block1)
        block1 = DepthwiseConv2D((Chans, 1), use_bias = False, padding='valid', depth_multiplier = n_sf[1], activation = 'linear',
                                 depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1),
                                name = 'Spatial_filter_1')(block1)
        block1 = BatchNormalization()(block1)
        block1 = Activation('elu')(block1)

        #================================

        block2 = Conv2D(n_ff[1], (1, 32), use_bias = False, activation = 'linear', padding='same',
                       name = 'Spectral_filter_2')(Input_block)
        block2 = BatchNormalization()(block2)
        block2 = DepthwiseConv2D((Chans, 1), use_bias = False, padding='valid', depth_multiplier = n_sf[2], activation = 'linear',
                                 depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1),
                                name = 'Spatial_filter_2')(block2)
        block2 = BatchNormalization()(block2)
        block2 = Activation('elu')(block2)

        #================================

        block3 = Conv2D(n_ff[2], (1, 64), use_bias = False, activation = 'linear', padding='same',
                       name = 'Spectral_filter_3')(Input_block)
        block3 = BatchNormalization()(block3)
        block3 = DepthwiseConv2D((Chans, 1), use_bias = False, padding='valid', depth_multiplier = n_sf[3], activation = 'linear',
                                 depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1), 
                                 name = 'Spatial_filter_3')(block3)
        block3 = BatchNormalization()(block3)
        block3 = Activation('elu')(block3)

        #================================

        block = Concatenate(axis = -1)([block1, block2, block3]) 

        #================================

        block = AveragePooling2D((1, 4))(block)
        block_in = Dropout(drop_rate)(block)

        #================================

        paddings = tf.constant([[0,0], [0,0], [3,0], [0,0]])
        block = tf.pad(block_in, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 1))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block = tf.pad(block, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 1))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block_out = Add()([block_in, block])


        paddings = tf.constant([[0,0], [0,0], [6,0], [0,0]])
        block = tf.pad(block_out, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 2))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block = tf.pad(block, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 2))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block_out = Add()([block_out, block])


        paddings = tf.constant([[0,0], [0,0], [12,0], [0,0]])
        block = tf.pad(block_out, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 4))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block = tf.pad(block, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 4))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block_out = Add()([block_out, block]) 


        paddings = tf.constant([[0,0], [0,0], [24,0], [0,0]])
        block = tf.pad(block_out, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 8))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block = tf.pad(block, paddings, "CONSTANT")
        block = DepthwiseConv2D((1,4), padding="valid", depth_multiplier=1, dilation_rate=(1, 8))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = Dropout(drop_rate)(block)
        block_out = Add()([block_out, block]) 

        #================================

        block = block_out

        #================================

        block = Conv2D(28, (1,1))(block)
        block = BatchNormalization()(block)
        block = Activation('elu')(block)
        block = AveragePooling2D((4,1), data_format='Channels_first')(block)
        block = Dropout(drop_rate)(block) 
        embedded = Flatten()(block)

        out = Dense(out_class, activation = 'softmax', kernel_constraint = max_norm(0.25))(embedded)
    
    #========================================================================================
    if out_type == 'multi':

        return Model(inputs = Input_block, outputs = [out, embedded])

    else:

        return Model(inputs = Input_block, outputs = out)

### Colab GPU

Test Colab GPU:

In [ ]:
!nvidia-smi

Thu Aug 11 09:19:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Read data

We need to read two signal data for now(only alphabet signals):
* Original signals
* Segment signals

In [3]:
from ml_functions import prepare_signals

In [4]:
def read_signal_data(feature_path):

    # feature_path = "./data/EEG_features_Lintao/eeg_signals/"
    alphabet_vision = pd.read_csv(feature_path+"aat_vision.csv", header=0, index_col=0)
    alphabet_imagination = pd.read_csv(feature_path+"aat_img.csv", header=0, index_col=0)

    return alphabet_vision, alphabet_imagination

In [6]:
feature_path = "/content/gdrive/MyDrive/UMONS/Notebooks_DL/data/EEG_features_Lintao/eeg_signals/"
aat_vis_signal, aat_img_signal = read_signal_data(feature_path)
aat_vis = prepare_signals(aat_vis_signal)
aat_img = prepare_signals(aat_img_signal)

In [7]:
aat_vis.shape

(832, 346, 16)

In [8]:
aat_vis = np.transpose(aat_vis,axes=(0,2,1))
aat_img = np.transpose(aat_img,axes=(0,2,1))
labels = np.array(aat_vis_signal['label_index'])

In [9]:
aat_vis.shape

(832, 16, 346)

For the segment data, I need to read the original signals and split them in real time. There is a problem with the previously stored split signal and cannot be read; also because of the real-time read split, I will keep the signal length consistent when reading:

In [10]:
# min 84  max 115
def read_seg_signal_data(feature_path, length_sig=85, seg_piece=4):

    # feature_path = "/content/gdrive/MyDrive/UMONS/Notebooks_DL/data/EEG_features_Lintao/eeg_signals/"
    aat_vis_signal = pd.read_csv(feature_path+"aat_vision.csv", header=0, index_col=0)
    aat_img_signal = pd.read_csv(feature_path+"aat_img.csv", header=0, index_col=0)
    # asl_vision = pd.read_csv(feature_path+"asl_vision.csv", header=0, index_col=0)
    # asl_imagination = pd.read_csv(feature_path+"asl_img.csv", header=0, index_col=0)
    
    labels = np.array(aat_vis_signal['label_index'])
    ch_names=['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8','ch9', 'ch10',
          'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16']
    seg_labels = list()
    seg_aat_vis = list()
    seg_aat_img = list()

    for i in range(aat_vis_signal.shape[0]):
        current_vis = list()
        current_img = list()
        for name in ch_names:
            current_vis.append(np.array(eval(aat_vis_signal[name][i])))
            current_img.append(np.array(eval(aat_img_signal[name][i])))
        current_vis = np.array(current_vis)
        current_img = np.array(current_img)
        seg = int(current_vis.shape[-1]/seg_piece)
        seg_2 = int(current_img.shape[-1]/seg_piece)
        
        # keep our signal length same in order to feed into network
        seg_aat_vis.extend([current_vis[:, 0:length_sig], current_vis[:,seg:seg+length_sig], current_vis[:,seg*2:seg*2+length_sig],
                            current_vis[:, seg*3:seg*3+length_sig]])
        seg_aat_img.extend([current_img[:, 0:length_sig], current_img[:,seg_2:seg_2+length_sig], 
                            current_img[:,seg_2*2:seg_2*2+length_sig], current_img[:,seg_2*3:seg_2*3+length_sig]])
        seg_labels.extend([labels[i], labels[i], labels[i], labels[i]])
        
    # [data size, channels, signal lengths]
    return np.array(seg_aat_vis), np.array(seg_aat_img), np.array(seg_labels)

In [64]:
aat_vis_signals, aat_img_signals, seg_labels = read_seg_signal_data(feature_path)

In [12]:
aat_vis_signals.shape

(3328, 16, 85)

In [13]:
seg_labels.shape

(3328,)

## Binary classification

### Task 1 Binary classification task using the original simple network.

Our task 1 has been tested before(with original signals), we can find that simple neural networks such as RNN and CNN cannot complete our binary classification task (distinguish imagination from vision, distinguish sign language and alphabet signals), here we first use augmentation segment signals to retest our task 1-(imagination VS vision).

In [ ]:
# Y = to_categorical(np.concatenate((seg_labels, seg_labels), axis=0))
Y = to_categorical(np.array([*[0 for i in range(832*4)], *[1 for i in range(832*4)]]))
X = np.concatenate((aat_vis_signals, aat_img_signals), axis=0)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
x_train.shape

(4659, 16, 85)

In [ ]:
y_train.shape

(4659, 2)

RNN LSTM GRU

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(16, 85)))
model.add(Dense(2,activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 3s - loss: 0.7072 - accuracy: 0.5048 - val_loss: 0.6996 - val_accuracy: 0.4972 - 3s/epoch - 21ms/step
Epoch 2/300
146/146 - 1s - loss: 0.6948 - accuracy: 0.5130 - val_loss: 0.6931 - val_accuracy: 0.4977 - 948ms/epoch - 6ms/step
Epoch 3/300
146/146 - 1s - loss: 0.6975 - accuracy: 0.5027 - val_loss: 0.6993 - val_accuracy: 0.5038 - 820ms/epoch - 6ms/step
Epoch 4/300
146/146 - 1s - loss: 0.6967 - accuracy: 0.4967 - val_loss: 0.6933 - val_accuracy: 0.5028 - 805ms/epoch - 6ms/step
Epoch 5/300
146/146 - 1s - loss: 0.6971 - accuracy: 0.5001 - val_loss: 0.7012 - val_accuracy: 0.4972 - 938ms/epoch - 6ms/step
Epoch 6/300
146/146 - 1s - loss: 0.6957 - accuracy: 0.4915 - val_loss: 0.6922 - val_accuracy: 0.5063 - 947ms/epoch - 6ms/step
Epoch 7/300
146/146 - 1s - loss: 0.6967 - accuracy: 0.4971 - val_loss: 0.6919 - val_accuracy: 0.5053 - 800ms/epoch - 5ms/step
Epoch 8/300
146/146 - 1s - loss: 0.6965 - accuracy: 0.4887 - val_loss: 0.6953 - val_accuracy: 0.4977 - 808ms/epoch - 6ms

In notebook 06.1, the training accuracy rate is about 70%, so for task 1, both the original signal and the segmented signal perform in general.

So test CNN:

In [ ]:
model = Sequential()
model.add(Conv1D(16,4, input_shape=(16, 85), activation='relu'))
model.add(Conv1D(4,2, activation='relu'))
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 7s - loss: 37.8020 - accuracy: 0.4997 - val_loss: 0.6950 - val_accuracy: 0.5048 - 7s/epoch - 47ms/step
Epoch 2/300
146/146 - 1s - loss: 0.6940 - accuracy: 0.4980 - val_loss: 0.6932 - val_accuracy: 0.5058 - 580ms/epoch - 4ms/step
Epoch 3/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.4767 - val_loss: 0.6932 - val_accuracy: 0.4942 - 564ms/epoch - 4ms/step
Epoch 4/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4942 - 603ms/epoch - 4ms/step
Epoch 5/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4942 - 575ms/epoch - 4ms/step
Epoch 6/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.4965 - val_loss: 0.6932 - val_accuracy: 0.4942 - 541ms/epoch - 4ms/step
Epoch 7/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4942 - 561ms/epoch - 4ms/step
Epoch 8/300
146/146 - 1s - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4942 - 569ms/epoch - 4m

The CNN performance is also very poor, we can be sure that the simple network can not even complete our binary classification, in the next small chapter, we will test whether our complex model can complete this task:

(Note - the model structure I will print in 26 classes)

### Task 2 Binary classification task using complex neural network.



In [ ]:
# configure the EEGNet-8,2,16 model with kernel length of 85 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 2, Chans = 16, Samples = 85, 
               dropoutRate = 0.5, kernLength = 62, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

# compile the model and set the optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])   

history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 2s - loss: 0.6963 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.5103 - 2s/epoch - 11ms/step
Epoch 2/300
146/146 - 1s - loss: 0.6950 - accuracy: 0.4965 - val_loss: 0.6935 - val_accuracy: 0.4942 - 745ms/epoch - 5ms/step
Epoch 3/300
146/146 - 1s - loss: 0.6942 - accuracy: 0.4915 - val_loss: 0.6933 - val_accuracy: 0.4937 - 734ms/epoch - 5ms/step
Epoch 4/300
146/146 - 1s - loss: 0.6938 - accuracy: 0.4930 - val_loss: 0.6935 - val_accuracy: 0.4977 - 727ms/epoch - 5ms/step
Epoch 5/300
146/146 - 1s - loss: 0.6937 - accuracy: 0.5020 - val_loss: 0.6934 - val_accuracy: 0.4972 - 755ms/epoch - 5ms/step
Epoch 6/300
146/146 - 1s - loss: 0.6938 - accuracy: 0.4913 - val_loss: 0.6932 - val_accuracy: 0.5038 - 744ms/epoch - 5ms/step
Epoch 7/300
146/146 - 1s - loss: 0.6935 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.4947 - 737ms/epoch - 5ms/step
Epoch 8/300
146/146 - 1s - loss: 0.6937 - accuracy: 0.4950 - val_loss: 0.6932 - val_accuracy: 0.4997 - 828ms/epoch - 6ms

In [ ]:
model = DeepConvNet(nb_classes = 2, Chans = 16, Samples = 85)

# compile the model and set the optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])   

history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 3s - loss: 0.7315 - accuracy: 0.4896 - val_loss: 0.6957 - val_accuracy: 0.4972 - 3s/epoch - 19ms/step
Epoch 2/300
146/146 - 1s - loss: 0.7101 - accuracy: 0.4870 - val_loss: 0.6944 - val_accuracy: 0.4922 - 956ms/epoch - 7ms/step
Epoch 3/300
146/146 - 1s - loss: 0.7009 - accuracy: 0.4943 - val_loss: 0.6940 - val_accuracy: 0.4917 - 1s/epoch - 8ms/step
Epoch 4/300
146/146 - 1s - loss: 0.7020 - accuracy: 0.4992 - val_loss: 0.6945 - val_accuracy: 0.4972 - 958ms/epoch - 7ms/step
Epoch 5/300
146/146 - 1s - loss: 0.6977 - accuracy: 0.4965 - val_loss: 0.6933 - val_accuracy: 0.4947 - 935ms/epoch - 6ms/step
Epoch 6/300
146/146 - 1s - loss: 0.6976 - accuracy: 0.4937 - val_loss: 0.6972 - val_accuracy: 0.5038 - 1s/epoch - 7ms/step
Epoch 7/300
146/146 - 1s - loss: 0.6985 - accuracy: 0.4943 - val_loss: 0.6958 - val_accuracy: 0.4962 - 919ms/epoch - 6ms/step
Epoch 8/300
146/146 - 1s - loss: 0.6984 - accuracy: 0.5005 - val_loss: 0.6973 - val_accuracy: 0.5038 - 921ms/epoch - 6ms/step


In [ ]:
model = Network(Chans = 16, Samples = 85, Net_name=1,drop_rate=0.5, out_class=2)

# compile the model and set the optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])   

history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 5s - loss: 0.7711 - accuracy: 0.4962 - val_loss: 0.6971 - val_accuracy: 0.5058 - 5s/epoch - 36ms/step
Epoch 2/300
146/146 - 1s - loss: 0.7341 - accuracy: 0.4932 - val_loss: 0.6945 - val_accuracy: 0.4937 - 1s/epoch - 9ms/step
Epoch 3/300
146/146 - 1s - loss: 0.7155 - accuracy: 0.4932 - val_loss: 0.6964 - val_accuracy: 0.4842 - 1s/epoch - 9ms/step
Epoch 4/300
146/146 - 1s - loss: 0.7084 - accuracy: 0.4969 - val_loss: 0.6936 - val_accuracy: 0.5038 - 1s/epoch - 10ms/step
Epoch 5/300
146/146 - 1s - loss: 0.7034 - accuracy: 0.5065 - val_loss: 0.6936 - val_accuracy: 0.4912 - 1s/epoch - 9ms/step
Epoch 6/300
146/146 - 1s - loss: 0.7001 - accuracy: 0.5149 - val_loss: 0.6937 - val_accuracy: 0.5068 - 1s/epoch - 10ms/step
Epoch 7/300
146/146 - 1s - loss: 0.7014 - accuracy: 0.4945 - val_loss: 0.6941 - val_accuracy: 0.4922 - 1s/epoch - 10ms/step
Epoch 8/300
146/146 - 1s - loss: 0.6993 - accuracy: 0.4992 - val_loss: 0.6937 - val_accuracy: 0.4942 - 1s/epoch - 10ms/step
Epoch 9/300

Although we did not test all 7 complex models, from the performance of the current three models, it is not necessary to test all network structures.

## 26 labels classification

### Task 3 Multi-classification task using the original simple network.

In [ ]:
Y = to_categorical(np.concatenate((seg_labels, seg_labels), axis=0))
X = np.concatenate((aat_vis_signals, aat_img_signals), axis=0)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
Counter(seg_labels)

Counter({1: 128,
         2: 128,
         3: 128,
         4: 128,
         5: 128,
         6: 128,
         7: 128,
         8: 128,
         9: 128,
         10: 128,
         11: 128,
         12: 128,
         13: 128,
         14: 128,
         15: 128,
         16: 128,
         17: 128,
         18: 128,
         19: 128,
         20: 128,
         21: 128,
         22: 128,
         23: 128,
         24: 128,
         25: 128,
         26: 128})

In [ ]:
y_train.shape

(4659, 27)

In [ ]:
y_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

RNN LSTM GRU

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(16, 85)))
model.add(Dense(27,activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 512)               1224704   
                                                                 
 dense_1 (Dense)             (None, 27)                13851     
                                                                 
Total params: 1,238,555
Trainable params: 1,238,555
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 3s - loss: 3.3329 - accuracy: 0.0412 - val_loss: 3.2814 - val_accuracy: 0.0466 - 3s/epoch - 20ms/step
Epoch 2/300
146/146 - 1s - loss: 3.2802 - accuracy: 0.0543 - val_loss: 3.2943 - val_accuracy: 0.0386 - 936ms/epoch - 6ms/step
Epoch 3/300
146/146 - 1s - loss: 3.2781 - accuracy: 0.0603 - val_loss: 3.2839 - val_accuracy: 0.0761 - 799ms/epoch - 5ms/step
Epoch 4/300
146/146 - 1s - loss: 3.2591 - accuracy: 0.0725 - val_loss: 3.2656 - val_accuracy: 0.0811 - 926ms/epoch - 6ms/step
Epoch 5/300
146/146 - 1s - loss: 3.2482 - accuracy: 0.0760 - val_loss: 3.2404 - val_accuracy: 0.0756 - 919ms/epoch - 6ms/step
Epoch 6/300
146/146 - 1s - loss: 3.2476 - accuracy: 0.0760 - val_loss: 3.2399 - val_accuracy: 0.0776 - 786ms/epoch - 5ms/step
Epoch 7/300
146/146 - 1s - loss: 3.2378 - accuracy: 0.0786 - val_loss: 3.2535 - val_accuracy: 0.0726 - 930ms/epoch - 6ms/step
Epoch 8/300
146/146 - 1s - loss: 3.2354 - accuracy: 0.0743 - val_loss: 3.2660 - val_accuracy: 0.0696 - 908ms/epoch - 6ms

We could find that the performance is very poor(acc < 10%), so does RNN, GRU, basic neural network cannot get progress in this task.

Try CNN:

In [ ]:
model = Sequential()
model.add(Conv1D(16,8, input_shape=(16, 85), activation='relu'))
model.add(Conv1D(8,4, activation='relu'))
model.add(Flatten())
model.add(Dense(27,activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 9, 16)             10896     
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 8)              520       
                                                                 
 flatten (Flatten)           (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 27)                1323      
                                                                 
Total params: 12,739
Trainable params: 12,739
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 5s - loss: 211.0462 - accuracy: 0.0337 - val_loss: 3.2912 - val_accuracy: 0.0376 - 5s/epoch - 37ms/step
Epoch 2/300
146/146 - 1s - loss: 3.2894 - accuracy: 0.0388 - val_loss: 3.2875 - val_accuracy: 0.0305 - 535ms/epoch - 4ms/step
Epoch 3/300
146/146 - 1s - loss: 3.2853 - accuracy: 0.0399 - val_loss: 3.2846 - val_accuracy: 0.0305 - 544ms/epoch - 4ms/step
Epoch 4/300
146/146 - 1s - loss: 3.2820 - accuracy: 0.0419 - val_loss: 3.2823 - val_accuracy: 0.0305 - 532ms/epoch - 4ms/step
Epoch 5/300
146/146 - 1s - loss: 3.2792 - accuracy: 0.0419 - val_loss: 3.2803 - val_accuracy: 0.0305 - 556ms/epoch - 4ms/step
Epoch 6/300
146/146 - 1s - loss: 3.2769 - accuracy: 0.0419 - val_loss: 3.2787 - val_accuracy: 0.0305 - 544ms/epoch - 4ms/step
Epoch 7/300
146/146 - 1s - loss: 3.2749 - accuracy: 0.0399 - val_loss: 3.2773 - val_accuracy: 0.0305 - 543ms/epoch - 4ms/step
Epoch 8/300
146/146 - 1s - loss: 3.2731 - accuracy: 0.0419 - val_loss: 3.2762 - val_accuracy: 0.0305 - 545ms/epoch - 4

Vey poor(acc < 5 %), try more complex models and NN!

### Task 4 Multi-classification tasks using complex neural networks.
#### EEGNet v1

In [ ]:
# configure the EEGNet-8,2,16 model with kernel length of 85 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 27, Chans = 16, Samples = 85, 
               dropoutRate = 0.5, kernLength = 62, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 85, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 85, 8)         496       
                                                                 
 batch_normalization (BatchN  (None, 16, 85, 8)        32        
 ormalization)                                                   
                                                                 
 depthwise_conv2d (Depthwise  (None, 1, 85, 16)        256       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 1, 85, 16)        64        
 hNormalization)                                                 
                                                             

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 16, epochs = 300, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/300
292/292 - 10s - loss: 3.2632 - accuracy: 0.0614 - val_loss: 3.2394 - val_accuracy: 0.0706 - 10s/epoch - 34ms/step
Epoch 2/300
292/292 - 1s - loss: 3.2187 - accuracy: 0.0805 - val_loss: 3.2099 - val_accuracy: 0.0641 - 1s/epoch - 4ms/step
Epoch 3/300
292/292 - 1s - loss: 3.1990 - accuracy: 0.0796 - val_loss: 3.1908 - val_accuracy: 0.0771 - 1s/epoch - 4ms/step
Epoch 4/300
292/292 - 1s - loss: 3.1842 - accuracy: 0.0835 - val_loss: 3.1795 - val_accuracy: 0.0826 - 1s/epoch - 4ms/step
Epoch 5/300
292/292 - 1s - loss: 3.1786 - accuracy: 0.0859 - val_loss: 3.1786 - val_accuracy: 0.0801 - 1s/epoch - 4ms/step
Epoch 6/300
292/292 - 1s - loss: 3.1587 - accuracy: 0.0921 - val_loss: 3.1610 - val_accuracy: 0.0771 - 1s/epoch - 4ms/step
Epoch 7/300
292/292 - 1s - loss: 3.1576 - accuracy: 0.0919 - val_loss: 3.1508 - val_accuracy: 0.0826 - 1s/epoch - 4ms/step
Epoch 8/300
292/292 - 1s - loss: 3.1531 - accuracy: 0.0938 - val_loss: 3.1441 - val_accuracy: 0.0876 - 1s/epoch - 4ms/step
Epoch 9/300
2

#### ShallowConvNet

In [ ]:
model = ShallowConvNet(nb_classes = 27, Chans = 16, Samples = 85)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16, 85, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 73, 40)        560       
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 73, 40)         25600     
                                                                 
 batch_normalization_3 (Batc  (None, 1, 73, 40)        160       
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 1, 73, 40)         0         
                                                                 
 average_pooling2d_2 (Averag  (None, 1, 6, 40)         0         
 ePooling2D)                                               

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 300, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/300
146/146 - 4s - loss: 3.7741 - accuracy: 0.0610 - val_loss: 3.2726 - val_accuracy: 0.0651 - 4s/epoch - 25ms/step
Epoch 2/300
146/146 - 1s - loss: 3.4143 - accuracy: 0.0668 - val_loss: 3.2147 - val_accuracy: 0.0646 - 587ms/epoch - 4ms/step
Epoch 3/300
146/146 - 1s - loss: 3.3824 - accuracy: 0.0771 - val_loss: 3.1927 - val_accuracy: 0.0751 - 577ms/epoch - 4ms/step
Epoch 4/300
146/146 - 1s - loss: 3.2982 - accuracy: 0.0805 - val_loss: 3.2022 - val_accuracy: 0.0761 - 595ms/epoch - 4ms/step
Epoch 5/300
146/146 - 1s - loss: 3.2924 - accuracy: 0.0824 - val_loss: 3.1749 - val_accuracy: 0.0761 - 583ms/epoch - 4ms/step
Epoch 6/300
146/146 - 1s - loss: 3.2523 - accuracy: 0.0798 - val_loss: 3.1730 - val_accuracy: 0.0756 - 594ms/epoch - 4ms/step
Epoch 7/300
146/146 - 1s - loss: 3.2462 - accuracy: 0.0807 - val_loss: 3.1743 - val_accuracy: 0.0681 - 585ms/epoch - 4ms/step
Epoch 8/300
146/146 - 1s - loss: 3.2422 - accuracy: 0.0869 - val_loss: 3.1720 - val_accuracy: 0.0916 - 566ms/epoch - 4ms

#### DeepConvNet

In [ ]:
model = DeepConvNet(nb_classes = 27, Chans = 16, Samples = 85)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16, 85, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 81, 25)        150       
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 81, 25)         10025     
                                                                 
 batch_normalization_4 (Batc  (None, 1, 81, 25)        100       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 1, 81, 25)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 40, 25)        0         
 )                                                         

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 3000, verbose = 2, validation_data=(x_test, y_test))

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 501/3000
146/146 - 1s - loss: 2.4453 - accuracy: 0.2015 - val_loss: 2.4097 - val_accuracy: 0.2063 - 851ms/epoch - 6ms/step
Epoch 502/3000
146/146 - 1s - loss: 2.4296 - accuracy: 0.2142 - val_loss: 2.3102 - val_accuracy: 0.2233 - 862ms/epoch - 6ms/step
Epoch 503/3000
146/146 - 1s - loss: 2.4477 - accuracy: 0.2061 - val_loss: 2.3439 - val_accuracy: 0.2238 - 996ms/epoch - 7ms/step
Epoch 504/3000
146/146 - 1s - loss: 2.4423 - accuracy: 0.2063 - val_loss: 2.3232 - val_accuracy: 0.1913 - 1s/epoch - 7ms/step
Epoch 505/3000
146/146 - 1s - loss: 2.4171 - accuracy: 0.2202 - val_loss: 2.2777 - val_accuracy: 0.2248 - 991ms/epoch - 7ms/step
Epoch 506/3000
146/146 - 1s - loss: 2.4236 - accuracy: 0.2078 - val_loss: 2.3183 - val_accuracy: 0.2138 - 861ms/epoch - 6ms/step
Epoch 507/3000
146/146 - 1s - loss: 2.4226 - accuracy: 0.2099 - val_loss: 2.2204 - val_accuracy: 0.2459 - 1s/epoch - 7ms/step
Epoch 508/3000
146/146 - 1s - loss: 2.4190 - accuracy: 0.2099 - val_loss: 2.

#### EEG-Inception

In [ ]:
model = Network(Chans = 16, Samples = 85, Net_name=1, drop_rate=0.5)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 16, 85, 1)]  0           []                               
                                                                                                  
 conv2d_8 (Conv2D)              (None, 16, 85, 8)    520         ['input_4[0][0]']                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 85, 8)    264         ['input_4[0][0]']                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 16, 85, 8)    136         ['input_4[0][0]']                
                                                                                            

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 500, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/500
146/146 - 4s - loss: 3.3854 - accuracy: 0.0455 - val_loss: 3.2492 - val_accuracy: 0.0796 - 4s/epoch - 28ms/step
Epoch 2/500
146/146 - 1s - loss: 3.2876 - accuracy: 0.0723 - val_loss: 3.2047 - val_accuracy: 0.0776 - 1s/epoch - 10ms/step
Epoch 3/500
146/146 - 1s - loss: 3.2609 - accuracy: 0.0745 - val_loss: 3.1916 - val_accuracy: 0.0651 - 1s/epoch - 9ms/step
Epoch 4/500
146/146 - 1s - loss: 3.2287 - accuracy: 0.0713 - val_loss: 3.1845 - val_accuracy: 0.0721 - 1s/epoch - 9ms/step
Epoch 5/500
146/146 - 1s - loss: 3.2168 - accuracy: 0.0758 - val_loss: 3.1788 - val_accuracy: 0.0811 - 1s/epoch - 9ms/step
Epoch 6/500
146/146 - 1s - loss: 3.2013 - accuracy: 0.0818 - val_loss: 3.1755 - val_accuracy: 0.0801 - 1s/epoch - 9ms/step
Epoch 7/500
146/146 - 1s - loss: 3.2015 - accuracy: 0.0824 - val_loss: 3.1714 - val_accuracy: 0.0766 - 1s/epoch - 10ms/step
Epoch 8/500
146/146 - 1s - loss: 3.1905 - accuracy: 0.0801 - val_loss: 3.1657 - val_accuracy: 0.0786 - 1s/epoch - 9ms/step
Epoch 9/500
1

#### EEGNet(v2)


In [ ]:
model = Network(Chans = 16, Samples = 85, Net_name=2,drop_rate=0.5)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16, 85, 1)]       0         
                                                                 
 Spectral_filter (Conv2D)    (None, 16, 85, 8)         512       
                                                                 
 batch_normalization_19 (Bat  (None, 16, 85, 8)        32        
 chNormalization)                                                
                                                                 
 Spatial_filter (DepthwiseCo  (None, 1, 85, 16)        256       
 nv2D)                                                           
                                                                 
 batch_normalization_20 (Bat  (None, 1, 85, 16)        64        
 chNormalization)                                                
                                                           

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 500, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/500
146/146 - 2s - loss: 3.2801 - accuracy: 0.0582 - val_loss: 3.2618 - val_accuracy: 0.0731 - 2s/epoch - 12ms/step
Epoch 2/500
146/146 - 1s - loss: 3.2275 - accuracy: 0.0801 - val_loss: 3.2153 - val_accuracy: 0.0736 - 705ms/epoch - 5ms/step
Epoch 3/500
146/146 - 1s - loss: 3.2031 - accuracy: 0.0861 - val_loss: 3.1963 - val_accuracy: 0.0786 - 697ms/epoch - 5ms/step
Epoch 4/500
146/146 - 1s - loss: 3.1839 - accuracy: 0.0861 - val_loss: 3.1758 - val_accuracy: 0.0841 - 670ms/epoch - 5ms/step
Epoch 5/500
146/146 - 1s - loss: 3.1722 - accuracy: 0.0906 - val_loss: 3.1602 - val_accuracy: 0.0796 - 671ms/epoch - 5ms/step
Epoch 6/500
146/146 - 1s - loss: 3.1598 - accuracy: 0.0936 - val_loss: 3.1521 - val_accuracy: 0.0906 - 674ms/epoch - 5ms/step
Epoch 7/500
146/146 - 1s - loss: 3.1455 - accuracy: 0.0974 - val_loss: 3.1476 - val_accuracy: 0.0806 - 672ms/epoch - 5ms/step
Epoch 8/500
146/146 - 1s - loss: 3.1404 - accuracy: 0.0979 - val_loss: 3.1334 - val_accuracy: 0.0851 - 670ms/epoch - 5ms

#### EEG-TCNet

In [ ]:
model = Network(Chans = 16, Samples = 85, Net_name=3, drop_rate=0.5)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 16, 85, 1)]  0           []                               
                                                                                                  
 Spectral_filter (Conv2D)       (None, 16, 85, 8)    512         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_22 (BatchN  (None, 16, 85, 8)   32          ['Spectral_filter[0][0]']        
 ormalization)                                                                                    
                                                                                                  
 Spatial_filter (DepthwiseConv2  (None, 1, 85, 16)   256         ['batch_normalization_22[0]

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 500, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/500
146/146 - 5s - loss: 3.3404 - accuracy: 0.0444 - val_loss: 3.2709 - val_accuracy: 0.0791 - 5s/epoch - 31ms/step
Epoch 2/500
146/146 - 2s - loss: 3.2675 - accuracy: 0.0685 - val_loss: 3.2112 - val_accuracy: 0.0736 - 2s/epoch - 10ms/step
Epoch 3/500
146/146 - 1s - loss: 3.2219 - accuracy: 0.0741 - val_loss: 3.1918 - val_accuracy: 0.0821 - 1s/epoch - 10ms/step
Epoch 4/500
146/146 - 1s - loss: 3.2102 - accuracy: 0.0859 - val_loss: 3.1784 - val_accuracy: 0.0831 - 1s/epoch - 10ms/step
Epoch 5/500
146/146 - 2s - loss: 3.1932 - accuracy: 0.0841 - val_loss: 3.1709 - val_accuracy: 0.0791 - 2s/epoch - 10ms/step
Epoch 6/500
146/146 - 1s - loss: 3.1888 - accuracy: 0.0813 - val_loss: 3.1597 - val_accuracy: 0.0836 - 1s/epoch - 10ms/step
Epoch 7/500
146/146 - 2s - loss: 3.1790 - accuracy: 0.0841 - val_loss: 3.1653 - val_accuracy: 0.0801 - 2s/epoch - 11ms/step
Epoch 8/500
146/146 - 1s - loss: 3.1769 - accuracy: 0.0846 - val_loss: 3.1534 - val_accuracy: 0.0776 - 1s/epoch - 10ms/step
Epoch 9/

#### EEG-ITNet

In [ ]:
model = Network(Chans = 16, Samples = 85, Net_name=4, drop_rate=0.3)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 16, 85, 1)]  0           []                               
                                                                                                  
 Spectral_filter_1 (Conv2D)     (None, 16, 85, 2)    32          ['input_7[0][0]']                
                                                                                                  
 Spectral_filter_2 (Conv2D)     (None, 16, 85, 4)    128         ['input_7[0][0]']                
                                                                                                  
 Spectral_filter_3 (Conv2D)     (None, 16, 85, 8)    512         ['input_7[0][0]']                
                                                                                            

In [ ]:
fittedModel = model.fit(x_train, y_train, batch_size = 32, epochs = 500, verbose = 2, validation_data=(x_test, y_test))

Epoch 1/500
146/146 - 29s - loss: 3.2638 - accuracy: 0.0670 - val_loss: 3.2475 - val_accuracy: 0.0711 - 29s/epoch - 196ms/step
Epoch 2/500
146/146 - 1s - loss: 3.1982 - accuracy: 0.0835 - val_loss: 3.1936 - val_accuracy: 0.0811 - 1s/epoch - 10ms/step
Epoch 3/500
146/146 - 2s - loss: 3.1757 - accuracy: 0.0863 - val_loss: 3.1686 - val_accuracy: 0.0816 - 2s/epoch - 10ms/step
Epoch 4/500
146/146 - 2s - loss: 3.1567 - accuracy: 0.0895 - val_loss: 3.1441 - val_accuracy: 0.0916 - 2s/epoch - 10ms/step
Epoch 5/500
146/146 - 2s - loss: 3.1431 - accuracy: 0.1000 - val_loss: 3.1414 - val_accuracy: 0.0951 - 2s/epoch - 11ms/step
Epoch 6/500
146/146 - 2s - loss: 3.1313 - accuracy: 0.0947 - val_loss: 3.1427 - val_accuracy: 0.0941 - 2s/epoch - 11ms/step
Epoch 7/500
146/146 - 2s - loss: 3.1223 - accuracy: 0.0987 - val_loss: 3.1258 - val_accuracy: 0.0976 - 2s/epoch - 10ms/step
Epoch 8/500
146/146 - 2s - loss: 3.1095 - accuracy: 0.0953 - val_loss: 3.1088 - val_accuracy: 0.1002 - 2s/epoch - 11ms/step
Epoch

From the performance of the above model, the most promising network only has an accuracy rate of 20%, which is far inferior to the result obtained by machine learning. In our task, deep learning are particularly bad compare with feature extraction + machine learning effect.

Of course, this is kind of normal. From another point of view, we can extract 86 features from one channel. These features cover all aspects of the signal, and some features also have high computational complexity.

Of course, this does not mean that deep learning is not good at such multi-classification tasks. It can only be said that a more complex network structure, or models such as transformers, may help us obtain better classification performance.

(Note that our signal is not filtered noise, and it may be better to use autoencoder to filter noise before classification)

## Imagined character recognition through EEG signals using deep convolutional neural network

### Task 5 Test the neural network model of the BP+CNN.

Morlet wavelet transform(Hz)

- delta 0.5-4
- theta 4-8  
- alpha 8-12  
- beta 12-30
- gamma 30-40

For convenience, the DWT directly used here is not exactly the same as the morlet wavelet mentioned in the original paper:

In [20]:
import pywt
from scipy import signal

def relativePower(X):
    Ca5, Cd5, Cd4, Cd3, Cd2, Cd1 = pywt.wavedec(X, wavelet='db4', level=5)
    EA5 = sum([i*i for i in Ca5])
    ED5 = sum([i*i for i in Cd5])
    ED4 = sum([i*i for i in Cd4])
    ED3 = sum([i*i for i in Cd3])
    ED2 = sum([i*i for i in Cd2])
    ED1 = sum([i*i for i in Cd1])
    E = EA5 + ED5 + ED4 + ED3 + ED2 + ED1
    pEA5 = EA5/E
    pED5 = ED5/E
    pED4 = ED4/E
    pED3 = ED3/E
    pED2 = ED2/E
    pED1 = ED1/E
    return [pEA5, pED5, pED4, pED3, pED2, pED1]

In [65]:
aat_vis_signals.shape

(3328, 16, 85)

In [66]:
bp_aat_vis = list()
for i in range(3328):
  current = list()
  for j in range(16):
    current.extend(relativePower(aat_vis_signals[i][j]))
  bp_aat_vis.append(np.array([np.array(current)]))

bp_aat_vis = np.array(bp_aat_vis)

bp_aat_img = list()
for i in range(3328):
  current = list()
  for j in range(16):
    current.extend(relativePower(aat_img_signals[i][j]))
  bp_aat_img.append(np.array([np.array(current)]))

bp_aat_img = np.array(bp_aat_img)

/usr/local/lib/python3.7/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


In [67]:
bp_aat_vis.shape

(3328, 1, 96)

In [68]:
seg_labels = [item -1 for item in seg_labels]

In [69]:
Y = to_categorical(np.concatenate((seg_labels, seg_labels), axis=0))
X = np.transpose(np.concatenate((bp_aat_vis, bp_aat_img), axis=0),axes=(0,2,1))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [70]:
len(y_train[0])

26

In [71]:
model = Sequential()
model.add(Conv1D(2,2, input_shape=(96, 1),padding='same',activation='relu'))
model.add(Conv1D(4,4, padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(26,activation='softmax'))

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_24 (Conv1D)          (None, 96, 2)             6         
                                                                 
 conv1d_25 (Conv1D)          (None, 96, 4)             36        
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 48, 4)            0         
 g1D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 192)               0         
                                                                 
 dense_44 (Dense)            (None, 64)                12352     
                                                                 
 dense_45 (Dense)            (None, 64)                4160      
                                                     

In [73]:
history = model.fit(x_train,y_train,batch_size=32,epochs=300,verbose=2,validation_data=(x_test,y_test))

Epoch 1/300
146/146 - 11s - loss: 3.2642 - accuracy: 0.0386 - val_loss: 3.2591 - val_accuracy: 0.0325 - 11s/epoch - 74ms/step
Epoch 2/300
146/146 - 1s - loss: 3.2582 - accuracy: 0.0397 - val_loss: 3.2598 - val_accuracy: 0.0290 - 561ms/epoch - 4ms/step
Epoch 3/300
146/146 - 1s - loss: 3.2578 - accuracy: 0.0425 - val_loss: 3.2606 - val_accuracy: 0.0290 - 574ms/epoch - 4ms/step
Epoch 4/300
146/146 - 1s - loss: 3.2577 - accuracy: 0.0425 - val_loss: 3.2612 - val_accuracy: 0.0290 - 589ms/epoch - 4ms/step
Epoch 5/300
146/146 - 1s - loss: 3.2575 - accuracy: 0.0425 - val_loss: 3.2619 - val_accuracy: 0.0290 - 576ms/epoch - 4ms/step
Epoch 6/300
146/146 - 1s - loss: 3.2574 - accuracy: 0.0395 - val_loss: 3.2623 - val_accuracy: 0.0290 - 547ms/epoch - 4ms/step
Epoch 7/300
146/146 - 1s - loss: 3.2574 - accuracy: 0.0425 - val_loss: 3.2627 - val_accuracy: 0.0290 - 571ms/epoch - 4ms/step
Epoch 8/300
146/146 - 1s - loss: 3.2573 - accuracy: 0.0425 - val_loss: 3.2631 - val_accuracy: 0.0290 - 553ms/epoch - 4

This effect is very different from the paper. Although the method is not exactly the same, the result should not have such a large error. There may be some problems elsewhere.(such as data quality, noise,signal time etc.)